# 03 Converting JSON to Tabular Data

In [ ]:
import json
import subprocess
import zipfile

from pathlib import Path

import pyarrow as pa
import polars as pl

from tqdm import tqdm

from timing import setup

setup()

root = Path("data/")

In [ ]:
subprocess.run(["cargo", "build", "-p", "json_to_arrow", "--release"], check=True)

In [ ]:
schema = {
    "name": pa.string(),
    "collaborative": pa.string(),
    "pid": pa.int64(),
    "modified_at": pa.date64(),
    "num_tracks": pa.int16(),
    "num_albums": pa.int16(),
    "num_followers": pa.int64(),
    "tracks": pa.list_(
        pa.struct(
            {
                "pos": pa.int16(),
                "artist_name": pa.string(),
                "track_uri": pa.string(),
                "artist_uri": pa.string(),
                "track_name": pa.string(),
                "album_uri": pa.string(),
                "duration_ms": pa.int64(),
                "album_name": pa.string(),
            }
        ),
    ),
}

In [ ]:
%%record-time convert-spotify-python

with zipfile.ZipFile(root / "spotify_million_playlist_dataset.zip", "r") as z:
    for i in range(1000):
        start = i * 1000 
        end = (i + 1) * 1000 - 1

        with z.open(f"data/mpd.slice.{start}-{end}.json", "r") as fobj:
            d = json.load(fobj)
        
        for playlist in d["playlists"]:
            playlist["modified_at"] = 1000 * playlist["modified_at"]
        
        table = pa.table({
            name: pa.array([pl[name] for pl in d["playlists"]], type=ty)
            for name, ty in schema.items()
        })
        chunk = pl.from_arrow(table)

In [ ]:
class DictBuilder:
    def __init__(self):
        self.index = {}
        self.indices = ArrayBuilder(pa.uint32())

    def append(self, value):
        index = self.index
        if value not in index:
            index[value] = len(index)

        self.indices.append(index[value])

    def build(self):
        values = pa.array(list(self.index), type=pa.large_utf8())
        self.index.clear()

        indices = self.indices.build()
        return pa.DictionaryArray.from_arrays(indices, values)


class _StructBuilder:
    _factory = None

    def __init__(self, **field_builders):
        self.field_builders = field_builders

    def append(self, value):
        for k, builder in self.field_builders.items():
            builder.append(value[k])

    def build(self):
        arrays = {k: builder.build() for k, builder in self.field_builders.items()}
        return self._factory(list(arrays.values()), list(arrays))


class RecordBatchBuilder(_StructBuilder):
    _factory = pa.record_batch


class StructBuilder(_StructBuilder):
    _factory = pa.StructArray.from_arrays


class ListBuilder:
    def __init__(self, item_builder):
        self.item_builder = item_builder
        self.offsets = [0]

    def append(self, value):
        for item in value:
            self.item_builder.append(item)

        self.offsets.append(self.offsets[-1] + len(value))

    def build(self):
        offsets = pa.array(self.offsets, type=pa.int64())
        self.offsets.clear()

        items = self.item_builder.build()
        return pa.LargeListArray.from_arrays(offsets, items)


class ArrayBuilder:
    def __init__(self, type):
        self.values = []
        self.type = type

        self._arrays = []

    def append(self, value):
        self.values.append(value)

        if len(self.values) > 1_024:
            self._build()

    def _build(self):
        self._arrays.append(pa.array(self.values, type=self.type))
        self.values.clear()

    def build(self):
        self._build()
        res = pa.concat_arrays(self._arrays)
        self._arrays.clear()
        return res

In [ ]:
%%record-time convert-spotify-python-builders

builder = RecordBatchBuilder(
    name=DictBuilder(),
    collaborative=DictBuilder(),
    pid=ArrayBuilder(pa.int64()),
    modified_at=ArrayBuilder(pa.date64()),
    num_tracks=ArrayBuilder(pa.int16()),
    num_albums=ArrayBuilder(pa.int16()),
    num_followers=ArrayBuilder(pa.int64()),
    tracks=ListBuilder(
        StructBuilder(
            pos=ArrayBuilder(pa.int16()),
            artist_name=DictBuilder(),
            track_uri=DictBuilder(),
            artist_uri=DictBuilder(),
            track_name=DictBuilder(),
            album_uri=DictBuilder(),
            album_name=DictBuilder(),
            duration_ms=ArrayBuilder(pa.int64()),
        ),
    ),
)

n = 1_000

with zipfile.ZipFile(root / "spotify_million_playlist_dataset.zip", "r") as z:
    for i in tqdm(range(n)):
        start = i * 1000 
        end = (i + 1) * 1000 - 1

        with z.open(f"data/mpd.slice.{start}-{end}.json", "r") as fobj:
            d = json.load(fobj)
        
        for pl in d["playlists"]:
            pl["modified_at"] = 1000 * pl["modified_at"]
            builder.append(pl)
        
batch = builder.build()

writer = pa.ipc.new_file("./data/spotify_million_playlist_dataset_py.ipc", batch.schema)
writer.write_batch(batch)
writer.close()

In [ ]:
%%record-time convert-spotify-rust
res = subprocess.run([
    "./target/release/json_to_arrow.exe",
    "./data/spotify_million_playlist_dataset.zip",
    "./data/spotify_million_playlist_dataset.ipc",
], check=True)

In [ ]:
%%record-time convert-spotify-rust-v2
res = subprocess.run([
    "./target/release/json_to_arrow_v2.exe",
    "./data/spotify_million_playlist_dataset.zip",
    "./data/spotify_million_playlist_dataset.ipc",
], check=True)

In [ ]:
%%record-time convert-spotify-rust-v3
res = subprocess.run([
    "./target/release/json_to_arrow_v3.exe",
    "./data/spotify_million_playlist_dataset.zip",
    "./data/spotify_million_playlist_dataset.ipc",
], check=True)

In [ ]:
%%record-time convert-spotify-rust-json-only
res = subprocess.run([
    "./target/release/json_to_arrow_json_only.exe",
    "./data/spotify_million_playlist_dataset.zip",
], check=True)

### Using the dataset

In [ ]:
df = pl.scan_ipc("data/spotify_million_playlist_dataset.ipc", memory_map=True)

In [ ]:
%%time
df.select(pl.col("collaborative").cast(pl.Float32).mean()).collect()

In [ ]:
%%time
counts = (
    df.select(
        pl.col("tracks")
        .arr.explode()
        .struct.field("artist_name")
        .value_counts()
        .alias("counts")
    )
    .select(
        [
            pl.col("counts").struct.field("artist_name"),
            pl.col("counts").struct.field("counts"),
        ]
    )
    .sort("counts")
    .tail(5)
    .collect()
)

counts

In [ ]:
%%time
df.select(pl.col("modified_at").dt.year().value_counts()).unnest("modified_at").sort(
    "counts"
).collect()